In [ ]:
using Random, Distributions
using Plots
# plotlyjs()
# using PlotlyJS
using DataFrames
using StatsBase
using DelimitedFiles
using LinearAlgebra
# using Distances
# using Symbolics
# using Latexify

# Problem Set 5 (Jonathan Fischer using Julia)

## 1) Langevin Dynamics of minimalist HP 

In [ ]:
function calc_dist(x1,x2, L=3.5) #calculate distance between a single pair of points along single component
    dx = x1-x2 
    dx - L*round(dx/L) #distance using minimum image convention 
end

In [ ]:
function calc_LJ(r, sigma = 1, epsilon = 0.25) # calculate Lennard_Jones potential for a pairwise distance
    4*epsilon*((sigma/r)^12 - (sigma/r)^6) 
end

In [ ]:
function calc_force(r, sigma = 1, epsilon = 0.25)
    48*epsilon*((sigma^12/r^13)-0.5*(sigma^6/r^7))
end